In [1]:
!pip install ftfy regex tqdm git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-yigel47r
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-yigel47r
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import json
import torch
import clip
import pandas as pd
from PIL import Image
from glob import glob
from tqdm import tqdm

In [3]:
# Load model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

100%|███████████████████████████████████████| 338M/338M [00:06<00:00, 51.2MiB/s]


In [6]:
import kagglehub
path = kagglehub.dataset_download("awsaf49/flickr30k-dataset")
# Đường dẫn chứa ảnh gốc
image_folder = "/kaggle/input/flickr30k-dataset/Images"

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os, shutil

# Tạo thư mục lưu ảnh
os.makedirs("data/train/General", exist_ok=True)
os.makedirs("data/test/General", exist_ok=True)

# Lấy danh sách ảnh
all_images = sorted(glob(os.path.join(image_folder, "*.jpg")))
print(f" Tổng số ảnh tìm thấy: {len(all_images)}")

# Copy ảnh: 200 test, 800 train
for i, path in enumerate(all_images):
    if i < 2000:
        shutil.copy(path, "data/test/General")
    elif i < 10000:
        shutil.copy(path, "data/train/General")

# Kiểm tra lại
print(" Done copying.")
print(" Train:", len(os.listdir("data/train/General")))
print(" Test:", len(os.listdir("data/test/General")))

 Tổng số ảnh tìm thấy: 31783
 Done copying.
 Train: 8000
 Test: 2000


In [8]:
caption_templates = [

    # People
    "A person standing on the street.",
    "A group of people walking together.",
    "A man riding a bicycle.",
    "A woman holding an umbrella.",
    "A child playing with a toy.",
    "A person sitting at a computer.",
    "A person jumping over a hurdle.",
    "A group of athletes running a race.",
    "A person holding a camera.",
    "A person painting a wall.",

    # Animals
    "A dog running in the park.",
    "A cat sitting on a couch.",
    "A bird flying in the sky.",
    "A horse galloping in a field.",
    "A group of ducks swimming in a lake.",
    "A monkey hanging from a tree.",
    "A dog playing with a ball.",
    "A cat looking out a window.",

    # Nature & Scenes
    "A beach with people sunbathing.",
    "A mountain landscape with snow.",
    "A forest full of trees.",
    "A city skyline at night.",
    "A street market full of people.",
    "A river flowing through the forest.",
    "A sunset over the ocean.",
    "A crowded park on a sunny day.",

    # Objects & Daily life
    "A plate of food on a table.",
    "A person cooking in a kitchen.",
    "A stack of books on a desk.",
    "A laptop on a coffee table.",
    "A person holding a smartphone.",
    "A backpack lying on the floor.",
    "A person pouring coffee into a cup.",
    "A close-up of a pizza.",

    # Vehicles
    "A car driving down a road.",
    "A group of motorcycles parked.",
    "A train at a station.",
    "A plane flying through the clouds.",
    "A person riding a scooter.",
    "A boat floating on a river.",

    # Education & Social
    "Students sitting in a classroom.",
    "A person reading a book.",
    "A teacher writing on a board.",
    "A group of people attending a meeting.",
    "Children playing on a playground.",
    "People clapping at a performance."
]

In [9]:
# Encode text trước
text_tokens = clip.tokenize(caption_templates).to(device)
with torch.no_grad():
    text_features = model.encode_text(text_tokens)
    text_features /= text_features.norm(dim=-1, keepdim=True)

In [11]:
# Hàm chọn caption cho ảnh
def get_caption(image_path):
    try:
        image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
        with torch.no_grad():
            image_features = model.encode_image(image)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            similarity = (image_features @ text_features.T).squeeze(0)
            best_idx = similarity.argmax().item()
        return caption_templates[best_idx]
    except Exception as e:
        print(f"Error loading {image_path}: {e}")
        return ""

In [12]:
# Duyệt ảnh và tạo pseudo-caption
folder = "data/test/General"
image_paths = sorted(glob(os.path.join(folder, "*.jpg")))

imgNum = 0
results = []
for path in tqdm(image_paths):
  imgNum += 1
  caption = get_caption(path)
  results.append({"image": path, "caption": caption})
  #print(f"Img {imgNum}: {os.path.basename(path)} → {caption}")

df = pd.DataFrame(results)
df.to_csv("pseudo_captions_clip.csv", index=False)


100%|██████████| 2000/2000 [00:34<00:00, 58.44it/s]
